In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os

import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']


# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



In [4]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

def load_data(filename):    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    #截掉前后数据
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    #滤波
    #return butter_worth(data , 0.5 , 30)

    return data

In [5]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq):
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/8/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
        
    #============
    #训练数据分段
    person_data_0 , person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    person_data_1 , person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    person_data_2 , person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    person_data_3 , person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    person_data_4 , person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    person_data_5 , person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    person_data_6 , person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    person_data_7 , person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    person_data_8 , person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    data = np.concatenate((person_data_0 , person_data_1 , person_data_2 ,
                           person_data_3 , person_data_4 , person_data_5 ,
                           person_data_6 , person_data_7 , person_data_8 
                        ))
    
    labels = np.concatenate((person_labels_0 , person_labels_1 , person_labels_2 ,
                             person_labels_3 , person_labels_4 , person_labels_5 ,
                             person_labels_6 , person_labels_7 , person_labels_8 
                        ))
    
    '''
    data维度 样本数量 , 256*3 , 16通道
    '''
    return data , labels

In [7]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input

Using TensorFlow backend.


# FC-AE

In [8]:
encoder_input = Input(shape = (256*3*16 , ))

#encoder
encoder = Dense(units=512 , activation='relu')(encoder_input)
encoder = Dense(units=256 , activation='relu')(encoder)
encoder = Dense(units=128 , activation='relu')(encoder)
encoder_output = Dense(units=64 , activation='relu')(encoder) #聚类需要使用的2维特征

#decoder
decoder = Dense(units=128 , activation='relu')(encoder_output)
decoder = Dense(units=256 , activation='relu')(decoder)
decoder = Dense(units=512 , activation='relu')(decoder)
decoder_output = Dense(units=256*3*16 , activation='relu')(decoder)


In [9]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# decision tree boosting

In [10]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.reshape(data , newshape=(-1 , 768*16))
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9416445623342176
0.946949602122016
0.9456233421750663
0.9429708222811671
0.9562334217506632
0.9496021220159151
0.9482758620689655
0.9615384615384616
0.9416445623342176
0.9522546419098143
0.9389920424403183
0.9416445623342176
0.9535809018567639
0.9456233421750663
0.9522546419098143
0.9588859416445623
0.9509283819628647
0.9416445623342176
0.9416445623342176
0.9442970822281167
freq =  6 average_acc =  0.9478116710875334 var_acc =  3.909740447058646e-05
freq =  7.5
0.9708222811671088
0.9376657824933687
0.9522546419098143
0.9496021220159151
0.9509283819628647
0.9535809018567639
0.9615384615384616
0.9602122015915119
0.9588859416445623
0.9562334217506632
0.9641909814323607
0.9615384615384616
0.9641909814323607
0.9522546419098143
0.9628647214854111
0.9549071618037135
0.9628647214854111
0.9615384615384616
0.9575596816976127
0.9655172413793104
freq =  7.5 average_acc =  0.9579575596816975 var_acc =  5.10275876140689e-05
freq =  8.5
0.9827586206896551
0.9774535809018567
0.980106100795

# svm boosting

In [10]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.reshape(data , newshape=(-1 , 768*16))
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.2002652519893899
0.28116710875331563
0.2413793103448276
0.17771883289124668
0.21220159151193635
0.21750663129973474
0.1724137931034483
0.21087533156498675
0.21883289124668434
0.20424403183023873
0.22679045092838196
0.2506631299734748
0.253315649867374
0.22944297082228116
0.22944297082228116
0.21220159151193635
0.17506631299734748
0.17506631299734748
0.17771883289124668
0.1843501326259947
freq =  6 average_acc =  0.21253315649867371 var_acc =  0.0008678515644238681
freq =  7.5
0.20822281167108753
0.2612732095490716
0.2427055702917772
0.2639257294429708
0.22679045092838196
0.2413793103448276
0.2480106100795756
0.2625994694960212
0.22148541114058357
0.1909814323607427
0.2453580901856764
0.1896551724137931
0.25596816976127323
0.2453580901856764
0.26790450928381965
0.253315649867374
0.22679045092838196
0.2639257294429708
0.22148541114058357
0.2440318302387268
freq =  7.5 average_acc =  0.2390583554376658 var_acc =  0.000521511267932653
freq =  8.5
0.3209549071618037
0.2997347480

# naive bayes boosting

In [10]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.reshape(data , newshape=(-1 , 768*16))
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.3594164456233422
0.43103448275862066
0.4230769230769231
0.376657824933687
0.35809018567639256
0.3116710875331565
0.33819628647214856
0.41644562334217505
0.3169761273209549
0.33819628647214856
0.29708222811671087
0.3753315649867374
0.31432360742705573
0.38726790450928383
0.38726790450928383
0.3793103448275862
0.35278514588859416
0.36074270557029176
0.3925729442970822
0.35278514588859416
freq =  6 average_acc =  0.3634615384615385 var_acc =  0.001334786531953366
freq =  7.5
0.32360742705570295
0.3448275862068966
0.34615384615384615
0.3541114058355438
0.3687002652519894
0.34350132625994695
0.3793103448275862
0.3793103448275862
0.27320954907161804
0.46419098143236076
0.4469496021220159
0.4403183023872679
0.41379310344827586
0.40981432360742703
0.4244031830238727
0.4323607427055703
0.48010610079575594
0.3793103448275862
0.40848806366047746
0.376657824933687
freq =  7.5 average_acc =  0.38945623342175073 var_acc =  0.002505342857544906
freq =  8.5
0.40450928381962864
0.4734748010

# CNN-AE

In [7]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , Deconv2D , UpSampling2D

Using TensorFlow backend.


In [8]:
encoder_input = Input(shape = (256*3 , 16 , 1))
#encoder
#VGG16结构
encoder = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder_input)
encoder = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
encoder = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
#new_add
encoder_output = Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder_output = MaxPooling2D(pool_size=(2, 2))(encoder_output)
#shape = encoder_output.get_shape().as_list()

encoder_output = Flatten()(encoder_output)
encoder_output = Dense(128 , activation='relu')(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(128 , activation='relu')(encoder_output)
decoder = Dense(48*1*32 , activation='relu')(decoder)
decoder = Reshape(target_shape=(48,1,32))(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder_output = Deconv2D(1, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)


In [9]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# decision tree boosting 
# 显存太小 4种频率

In [10]:
acc_s = []

for freq in [6]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9575596816976127
0.9641909814323607
0.9681697612732095
0.9681697612732095
0.9708222811671088
0.9761273209549072
0.9562334217506632
0.9761273209549072
0.9575596816976127
0.9761273209549072
0.9814323607427056
0.9615384615384616
0.96684350132626
0.9721485411140584
0.9761273209549072
0.9655172413793104
0.9655172413793104
0.9734748010610079
0.9721485411140584
0.9482758620689655
freq =  6 average_acc =  0.9677055702917773 var_acc =  6.653726544195758e-05
freq =  7.5


InternalError: cuDNN Backward Filter function launch failure : input shape([32,64,768,16]) filter shape([3,3,64,128])
	 [[Node: training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2DBackpropFilter = Conv2DBackpropFilter[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2D"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/conv2d_transpose_7/Relu_grad/ReluGrad, training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Shape, training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2DBackpropFilter-2-TransposeNHWCToNCHW-LayoutOptimizer)]]

In [10]:
acc_s = []

for freq in [7.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  7.5
0.9641909814323607
0.9721485411140584
0.9549071618037135
0.96684350132626
0.9588859416445623
0.9575596816976127
0.9694960212201591
0.9628647214854111
0.980106100795756
0.9615384615384616
0.9496021220159151
0.9628647214854111
0.9681697612732095
0.9588859416445623
0.9602122015915119
0.9681697612732095
0.9641909814323607
0.96684350132626
0.9588859416445623
0.9641909814323607
freq =  7.5 average_acc =  0.963527851458886 var_acc =  4.177542936346568e-05


In [10]:
acc_s = []

for freq in [8.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)
        
        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  8.5
0.9655172413793104
0.9708222811671088
0.9615384615384616
0.96684350132626
0.9628647214854111
0.9615384615384616
0.9694960212201591
0.9694960212201591
0.9734748010610079
0.9748010610079576
0.9655172413793104
0.9708222811671088
0.9721485411140584
0.9575596816976127
0.9774535809018567
0.9641909814323607
0.9708222811671088
0.980106100795756
0.9761273209549072
0.9761273209549072
freq =  8.5 average_acc =  0.9693633952254643 var_acc =  3.498582273849811e-05


In [10]:
acc_s = []

for freq in [10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  10
0.9721485411140584
0.9694960212201591
0.9628647214854111
0.9694960212201591
0.9588859416445623
0.9787798408488063
0.980106100795756
0.9641909814323607
0.9641909814323607
0.9721485411140584
0.9681697612732095
0.9575596816976127
0.9774535809018567
0.9641909814323607
0.96684350132626
0.9655172413793104
0.9721485411140584
0.9681697612732095
0.9655172413793104
0.9655172413793104
freq =  10 average_acc =  0.9681697612732094 var_acc =  3.447572275890208e-05


# 结束 
# ======

# svm boosting 
# 4种频率 显存太小

In [10]:
acc_s = []

for freq in [6]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.29973474801061006
0.2917771883289125
0.31432360742705573
0.31432360742705573
0.2427055702917772
0.259946949602122
0.25596816976127323
0.2612732095490716
0.27718832891246686
0.29442970822281167
0.3076923076923077
0.2493368700265252
0.2838196286472148
0.30238726790450926
0.22413793103448276
0.2639257294429708
0.28514588859416445
0.27055702917771884
0.27984084880636606
0.26657824933687
freq =  6 average_acc =  0.27725464190981436 var_acc =  0.000578431389793779
freq =  7.5


InternalError: cuDNN Backward Filter function launch failure : input shape([32,64,768,16]) filter shape([3,3,64,128])
	 [[Node: training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2DBackpropFilter = Conv2DBackpropFilter[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2D"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/conv2d_transpose_7/Relu_grad/ReluGrad, training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Shape, training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2DBackpropFilter-2-TransposeNHWCToNCHW-LayoutOptimizer)]]

In [10]:
acc_s = []

for freq in [7.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  7.5
0.2572944297082228
0.24005305039787797
0.26657824933687
0.23076923076923078
0.2572944297082228
0.21087533156498675
0.25596816976127323
0.2002652519893899
0.2546419098143236
0.22811671087533156
0.22281167108753316
0.22546419098143236
0.2427055702917772
0.20291777188328913
0.22148541114058357
0.26525198938992045
0.25596816976127323
0.22811671087533156
0.24005305039787797
0.21352785145888595
freq =  7.5 average_acc =  0.2360079575596817 var_acc =  0.0004037661209183208


In [10]:
acc_s = []

for freq in [8.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  8.5
0.40848806366047746
0.3169761273209549
0.4124668435013263
0.363395225464191
0.27586206896551724
0.3793103448275862
0.2957559681697613
0.44562334217506633
0.27320954907161804
0.2864721485411141
0.34615384615384615
0.3275862068965517
0.42572944297082227
0.3753315649867374
0.33156498673740054
0.29310344827586204
0.28779840848806365
0.39389920424403185
0.34615384615384615
0.3156498673740053
freq =  8.5 average_acc =  0.34502652519893895 var_acc =  0.002696718298165751


In [10]:
acc_s = []

for freq in [10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  10
0.33156498673740054
0.3660477453580902
0.2838196286472148
0.27718832891246686
0.28249336870026526
0.22944297082228116
0.3395225464190981
0.2891246684350133
0.3156498673740053
0.3010610079575597
0.3793103448275862
0.3222811671087533
0.32360742705570295
0.2572944297082228
0.35278514588859416
0.3156498673740053
0.33156498673740054
0.2625994694960212
0.3196286472148541
0.34084880636604775
freq =  10 average_acc =  0.3110742705570292 var_acc =  0.0013773534957679293


# 结束 
# =====

# naive bayes boosting 
# 4种频率 显存太小

In [14]:
acc_s = []

for freq in [6]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.519893899204244
0.493368700265252
0.47480106100795755
0.493368700265252
0.5490716180371353
0.5636604774535809
0.596816976127321
0.4920424403183024
0.4535809018567639
0.5570291777188329
0.45490716180371354
0.4880636604774536
0.5689655172413793
0.5278514588859416
0.48010610079575594
0.4761273209549072
0.46419098143236076
0.519893899204244
0.53315649867374
0.47877984084880637
freq =  6 average_acc =  0.5092838196286473 var_acc =  0.0016300332796262557
freq =  7.5


ResourceExhaustedError: OOM when allocating tensor with shape[32,64,768,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_5/Adam/gradients/conv2d_transpose_8/conv2d_transpose_grad/Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_5/Adam/gradients/conv2d_transpose_8/Relu_grad/ReluGrad, conv2d_transpose_8/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [10]:
acc_s = []

for freq in [7.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  7.5
0.5371352785145889
0.5305039787798409
0.5358090185676393
0.513262599469496
0.47480106100795755
0.5464190981432361
0.5358090185676393
0.4482758620689655
0.5636604774535809
0.5610079575596817
0.48408488063660476
0.46684350132625996
0.5822281167108754
0.5729442970822282
0.6047745358090185
0.5159151193633952
0.6061007957559682
0.48143236074270557
0.5464190981432361
0.4907161803713528
freq =  7.5 average_acc =  0.5299071618037134 var_acc =  0.0019454993351110616


In [10]:
acc_s = []

for freq in [8.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  8.5
0.5809018567639257
0.4403183023872679
0.46419098143236076
0.5251989389920424
0.473474801061008
0.4403183023872679
0.5424403183023873
0.3713527851458886
0.47745358090185674
0.4708222811671088
0.46816976127320953
0.44297082228116713
0.513262599469496
0.4588859416445623
0.4403183023872679
0.5172413793103449
0.5053050397877984
0.3885941644562334
0.3978779840848806
0.4920424403183024
freq =  8.5 average_acc =  0.4705570291777188 var_acc =  0.002579276572691006


In [10]:
acc_s = []

for freq in [10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  10
0.23209549071618038
0.17639257294429708
0.19230769230769232
0.25596816976127323
0.1896551724137931
0.2440318302387268
0.25596816976127323
0.1870026525198939
0.16710875331564987
0.22281167108753316
0.17904509283819628
0.22281167108753316
0.28514588859416445
0.1816976127320955
0.20291777188328913
0.15649867374005305
0.23076923076923078
0.21883289124668434
0.24005305039787797
0.2440318302387268
freq =  10 average_acc =  0.21425729442970826 var_acc =  0.0011476326084050407


# 结束 
# ==

# RNN-AE

In [7]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , LSTM

Using TensorFlow backend.


In [8]:
encoder_input = Input(shape = (256*3 , 16))

#encoder
encoder = LSTM(32 , return_sequences=True)(encoder_input)
encoder_output = Dense(64 , activation='relu')(encoder)
encoder_output = Flatten()(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(768*64 , activation='relu')(encoder_output)
decoder = Reshape(target_shape=(768 , 64))(decoder)
decoder = LSTM(32)(decoder)
decoder_output = Dense(256*3*16 , activation='relu')(decoder)
decoder_output = Reshape(target_shape=(256*3 , 16))(decoder_output)

In [9]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# decision tree boosting

In [19]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.29310344827586204
0.2440318302387268
0.21220159151193635
0.27586206896551724
0.25596816976127323
0.21220159151193635
0.28779840848806365
0.2546419098143236
0.2546419098143236
0.27984084880636606
0.20159151193633953
0.2718832891246684
0.253315649867374
0.1724137931034483
0.27055702917771884
0.2572944297082228
0.27851458885941643
0.2957559681697613
0.20822281167108753
0.22679045092838196
freq =  6 average_acc =  0.2503315649867374 var_acc =  0.0011167451751577788
freq =  7.5
0.27718832891246686
0.26657824933687
0.27055702917771884
0.23607427055702918
0.2493368700265252
0.22546419098143236
0.2625994694960212
0.2480106100795756
0.22413793103448276
0.21352785145888595
0.2612732095490716
0.24005305039787797
0.2506631299734748
0.25596816976127323
0.23872679045092837
0.23607427055702918
0.23607427055702918
0.253315649867374
0.2612732095490716
0.2546419098143236
freq =  7.5 average_acc =  0.24807692307692308 var_acc =  0.0002628729886230115
freq =  8.5
0.3594164456233422
0.327586206

# svm boosting 
# 4种频率 显存太小 

In [10]:
acc_s = []

for freq in [6]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.3063660477453581
0.3793103448275862
0.3037135278514589
0.3302387267904509
0.3952254641909814
0.3885941644562334
0.356763925729443
0.3448275862068966
0.32891246684350134
0.3660477453580902
0.3726790450928382
0.3116710875331565
0.32891246684350134
0.3806366047745358
0.3806366047745358
0.30238726790450926
0.34350132625994695
0.3103448275862069
0.4151193633952255
0.3620689655172414
freq =  6 average_acc =  0.3503978779840849 var_acc =  0.0011087814590970177


In [10]:
acc_s = []

for freq in [7.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  7.5
0.2440318302387268
0.27320954907161804
0.25596816976127323
0.2546419098143236
0.33554376657824936
0.27718832891246686
0.2718832891246684
0.26790450928381965
0.23342175066312998
0.23872679045092837
0.2625994694960212
0.246684350132626
0.259946949602122
0.28779840848806365
0.27984084880636606
0.2612732095490716
0.3010610079575597
0.2639257294429708
0.2612732095490716
0.246684350132626
freq =  7.5 average_acc =  0.26618037135278516 var_acc =  0.0005153944655911181


In [10]:
acc_s = []

for freq in [8.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  8.5
0.7281167108753316
0.6684350132625995
0.669761273209549
0.7228116710875332
0.669761273209549
0.6724137931034483
0.6604774535809018
0.6896551724137931
0.7095490716180372
0.7095490716180372
0.6856763925729443
0.6578249336870027
0.6644562334217506
0.610079575596817
0.6259946949602122
0.7692307692307693
0.6790450928381963
0.7374005305039788
0.7387267904509284
0.5490716180371353
freq =  8.5 average_acc =  0.6809018567639257 var_acc =  0.00235866712634297


In [10]:
acc_s = []

for freq in [10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  10
0.6710875331564987
0.7586206896551724
0.629973474801061
0.6472148541114059
0.6405835543766578
0.6671087533156499
0.6923076923076923
0.6631299734748011
0.6671087533156499
0.6684350132625995
0.6458885941644562
0.6816976127320955
0.7042440318302388
0.5928381962864722
0.6644562334217506
0.6870026525198939
0.746684350132626
0.7453580901856764
0.7108753315649867
0.6989389920424404
freq =  10 average_acc =  0.6791777188328914 var_acc =  0.0015976507257491434


# 结束
# ==

# naive bayes boosting

In [10]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.22679045092838196
0.253315649867374
0.1989389920424403
0.27718832891246686
0.28514588859416445
0.22413793103448276
0.28514588859416445
0.2718832891246684
0.15915119363395225
0.22811671087533156
0.25596816976127323
0.27055702917771884
0.18037135278514588
0.1856763925729443
0.2692307692307692
0.26790450928381965
0.16445623342175067
0.16047745358090185
0.2002652519893899
0.27055702917771884
freq =  6 average_acc =  0.231763925729443 var_acc =  0.0018979017301184137
freq =  7.5
0.27718832891246686
0.32891246684350134
0.24005305039787797
0.21618037135278514
0.26657824933687
0.23209549071618038
0.2413793103448276
0.27453580901856767
0.29045092838196285
0.2625994694960212
0.2440318302387268
0.23342175066312998
0.2917771883289125
0.2546419098143236
0.2838196286472148
0.2917771883289125
0.23076923076923078
0.2917771883289125
0.2493368700265252
0.27718832891246686
freq =  7.5 average_acc =  0.26392572944297077 var_acc =  0.0007609284523214827
freq =  8.5
0.3978779840848806
0.35013262